In [1]:
# Load in libraries

import warnings
warnings.filterwarnings('ignore')

#libraries for handling data
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

#label encoders
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()


#libraries for data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
import seaborn as sns
sns.set_palette('Set2')

#libaries for modelling
# Regression Modelling Algorithms
import statsmodels.api as sm
#from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor


In [2]:
#train = pd.read_csv("/Users/reejungkim/Documents/Git/Kaggle---House-Price/train.csv", index_col=0)
train = pd.read_csv("https://raw.githubusercontent.com/reejungkim/HousingPrice/master/train.csv", error_bad_lines=False)
train.head()

HTTPError: HTTP Error 404: Not Found

In [ ]:
#test = pd.read_csv('/Users/reejungkim/Documents/Git/Kaggle---House-Price/test.csv', index_col=0)
test = pd.read_csv("https://raw.githubusercontent.com/reejungkim/HousingPrice/master/test.csv", error_bad_lines=False)
test.head()

In [ ]:
test.shape, train.shape

train.describe().transpose()

In [ ]:
all_data = pd.concat((train,test))

In [ ]:
all_data.shape

## EDA

In [ ]:
all_data.info()

#### columns with highest missing value portions

In [ ]:
(all_data.isnull().sum()/ len(all_data)).sort_values(ascending=False)[:30]

#### colums whose missing portions are greater than 80%

In [ ]:
df_null = pd.DataFrame((all_data.isnull().sum()/ len(all_data)), columns={"%missing"})
df_null.loc[df_null['%missing']>=.8]

#### Question: Should we drop and not use the data where there're more than 80% of data missing(NA)?
Per data description, the NA for those four features above means zero (0) instead of missing. So, we should NOT drop them at this stage.

Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access
       
       

PoolQC: Pool quality
		
       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       NA	No Pool
       
       
       
Fence: Fence quality
		
       GdPrv	Good Privacy
       MnPrv	Minimum Privacy
       GdWo	Good Wood
       MnWw	Minimum Wood/Wire
       NA	No Fence
	
MiscFeature: Miscellaneous feature not covered in other categories
		
       Elev	Elevator
       Gar2	2nd Garage (if not described in garage section)
       Othr	Other
       Shed	Shed (over 100 SF)
       TenC	Tennis Court
       NA	None


# handling null values

In [ ]:
# Null values of Electrical and Total BsmtSF would be replaced by mode
for col in ['Electrical', 'RoofStyle', 'Neighborhood']:
    all_data[col] = all_data[col].fillna(all_data[col].mode()[0])

In [ ]:
all_data.info()

all_data.loc[pd.isnull(all_data.LotFrontage)]

In [ ]:
#all_data.iloc[:,all_data.dtypes==object]
a = all_data.select_dtypes(include=['object']).columns
a

In [ ]:
#replace all null values to 0
for col in a:
    all_data[col] = all_data[col].fillna('None')

In [ ]:
b = all_data.select_dtypes(include=['int64', 'float64']).columns
b

In [ ]:
all_data['LotFrontage'].dtype

In [ ]:
# Null values of rest of numerical types variables would better be replaced by median of neighborhood group
for col in b:
    all_data[col] = all_data.groupby('Neighborhood')[col].transform( lambda x: x.fillna(x.median()))
 

In [ ]:
all_data.head()

# vectorization/ Encoding

In [ ]:
for col in a:
    print('{}: {}'.format(col, all_data[col].unique()  ))

In [ ]:
#x = le.fit_transform(all_data['Alley'].unique())
#y = le.inverse_transform(x)
#list(x), list(y)

In [ ]:
#Encoding object type variables using label encoder
for col in a:
    all_data[col] = le.fit_transform(all_data[col])

In [ ]:
all_data.head()

In [ ]:
all_data.info()

# Scaler

In [ ]:
#choose scaler
scaler = RobustScaler()

In [ ]:
#scaler application
scaled_X = scaler.fit_transform(all_data.loc[:, all_data.columns!='SalePrice'].reset_index(drop=True))
scaled_Y = scaler.fit_transform( all_data[0:1460][['SalePrice']] )  

In [ ]:
scaled_Y

In [ ]:
sns.distplot(scaled_Y)

# set x_train, y_train, x_test, and y_test

In [ ]:
x_train = scaled_X[0:1460]
y_train = scaled_Y[0:1460]
x_test = scaled_X[1460::]
y_test = scaled_Y[1460::]

In [ ]:
#linear regression
#lm = LinearRegression()
#lm.fit(x_train, y_train)
#y_pred_lm = lm.predict(x_test)
#lm.score(x_train, y_train) 


lm = sm.OLS(y_train, x_train).fit() ## sm.OLS(output, input)
y_pred_lm = lm.predict(x_test)

# Print out the statistics
lm.summary()

In [ ]:
#random forest regression
randomForest = RandomForestRegressor()
randomForest.fit(x_train, y_train)
y_pred_randomForest = randomForest.predict(x_test)

print("Train score: %0.3f" % randomForest.score(x_train, y_train) )

In [ ]:
#support vector regression
svr = SVR()
svr.fit(x_train, y_train)
y_pred_svr = svr.predict(x_test)

print("Train score: %0.3f" % svr.score(x_train, y_train) )

In [ ]:
#gradient boosting regression
gradientBoosting = GradientBoostingRegressor()
gradientBoosting.fit(x_train, y_train)
y_pred_gradBoost = gradientBoosting.predict(x_test)

print("Train score: %0.3f" % gradientBoosting.score(x_train, y_train))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV

# First create the base model to tune
rfr = RandomForestRegressor() #RandomForestRegressor()


# Number of trees in random forest
n_estimators = [3000] #[int(x) for x in np.linspace(start = 500, stop = 1000, num = 3)]
# Number of features to consider at every split
max_features = ['sqrt'] #['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [6, 8 , 10]#[int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [8, 12, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [8, 16, 20]
# Method of selecting samples for training each tree
bootstrap = [True] #[True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rfr_random = RandomizedSearchCV(estimator = rfr, 
                               param_distributions = random_grid, 
                               #n_iter = 5, 
                               cv = 3, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)


#clf = GridSearchCV(RandomForestRegressor(), tuned_parameters, cv=5,n_jobs=-1, verbose=1)

In [ ]:
# Fit the random search model
history = rfr_random.fit(x_train, y_train)

In [ ]:
for name, value in history.history.items():
    plt.plot(value, label=name)
    
plt.legend()


In [ ]:
rfr_random.score(x_train, y_train)

In [ ]:
rfr_random.score(x_test, y_pred)

In [ ]:
rfr_random.best_params_

# inverse scaler

In [ ]:
# predictions
y_pred = rfr_random.predict(x_test)
y_pred

y_pred = y_pred_randomForest

In [ ]:
y_new_inverse = scaler.inverse_transform(y_pred.reshape(-1,1))

In [ ]:
y_new_inverse

In [ ]:
y_new_inverse = np.squeeze(y_new_inverse)

In [ ]:
y_new_inverse.shape

In [ ]:
all_data.reset_index()['Id'][1460::].values.shape

In [ ]:
Id = (all_data.reset_index()['Id'][1460::]).values
result = pd.DataFrame( { 'Id': Id , 'SalePrice': y_new_inverse} )

In [ ]:
result

In [ ]:
#Export result as a csv file
import os
os.getcwd()

result.to_csv( 'Submission.csv' , index = False )

# check feature importance

In [ ]:
features = all_data.loc[:, all_data.columns!='SalePrice'].columns
importances = randomForest.feature_importances_

In [ ]:
feat_importances = pd.Series(importances, index=features)

#feat_importances.sort_values(ascending=True).plot(kind='barh', figsize=(10, 20))
#feat_importances.nlargest(5).sort_values(ascending=True).plot(kind='barh')
feat_importances.nlargest(5).sort_values(ascending=True).plot(kind='barh', figsize=(4, 3))

In [ ]:
dfFeatures = pd.DataFrame(feat_importances.nlargest(5), columns=['importance'])
dfFeatures